In [79]:
import numpy as np 
import pandas as pd 
import json
from sklearn.feature_extraction.text import CountVectorizer
import nltk

In [80]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credit = pd.read_csv('tmdb_5000_credits.csv')

In [81]:
credit.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [82]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [83]:
#credit[credit['movie_id'] == movies[movies['original_title'] =='Avatar']['id'].values[0]]['cast'][0]

In [84]:
#s = credit[credit['movie_id'] == movies[movies['original_title'] =='Avatar']['id'].values[0]]['cast'][0]

In [85]:
#cast_lidt = json.loads(s)

In [86]:
#cast_lidt[0]['name']

**MERGE DATABASE**

In [87]:
movies = movies.merge(credit , on='title')

In [88]:
movies.shape

(4809, 23)

In [89]:
# geners , id , keywords , title , overview , cast , crew
movies = movies[['id' , 'title' , 'overview' , 'genres' , 'keywords' , 'cast' , 'crew']]

In [90]:
movies.head()

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


**DATA PREPROCESSING**

In [91]:
# Removing the null values and duplicated values 
movies.isnull().sum()   # So it has 3 null values in the overview column so that is not a big number so we can remove that without any tension
movies.dropna(inplace=True)
movies.isnull().sum()
movies.duplicated().sum()

np.int64(0)

In [92]:
movies.iloc[0]['genres']

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [93]:
movies[movies['genres'] == '[]'].value_counts().sum()

np.int64(28)

In [94]:
# Code Generation
# j = json.loads(movies.iloc[0]['genres'])
# l = []
# for i in j:
#     print(i['name'])
#     l.append(i['name'])

In [95]:
# Convert Geners Into List
def convert(obj):
    jsn = json.loads(obj)
    l = []
    for i in jsn:
        l.append(i['name'])
    return l 

In [96]:
movies['genres'] = movies['genres'].apply(convert)

In [97]:
# Convert Keywords into List 
movies.iloc[0]['keywords']
def convert_keywords(obj):
    l = []
    jsn = json.loads(obj)
    for i in jsn:
        l.append(i['name'])
    return l

movies['keywords'] = movies['keywords'].apply(convert_keywords)


In [98]:
# Convert Cast into List
def cast_convert(obj):
    l = []
    jsn = json.loads(obj)
    if len(jsn) > 0 and len(jsn) >=3:
        for index in range(3): 
            l.append(jsn[index]['name'])
    else:
        return l
    return l

movies['cast'] = movies['cast'].apply(cast_convert)

In [99]:
# Fetching Director from Crew Column
def fetch_director(obj):
    l = []
    jsn = json.loads(obj)
    for i in jsn:
        if i['job'] == 'Director':
             l.append(i['name'])
    return l

fetch_director(movies['crew'].iloc[123])
movies['crew'] = movies['crew'].apply(fetch_director)

In [100]:
# Now working on the overview to make the whole string as a List to concatinate with all the other lists 
movies['overview'] = movies['overview'].apply(lambda x : x.split())

In [101]:
movies.head()

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]


In [102]:
#Lets say above data has sam Worthington in case and same in creq but different name sam mendes so machine will confuse so we are trying to make it a single word so machine can understand so we are trying to remove the space from them

movies['genres'] = movies['genres'].apply(lambda x : [i.replace(" " , "") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x : [i.replace(" " , "") for i in x])
movies['cast'] = movies['cast'].apply(lambda x : [i.replace(" " , "") for i in x])
movies['crew'] = movies['crew'].apply(lambda x : [i.replace(" " , "") for i in x])

In [103]:
# Making all the processed column as a single list and make a new column as tags
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [104]:
# Made a new DataFrame 
new_df = movies.drop(columns=['overview' , 'cast','crew' , 'genres', 'keywords'])
new_df.head()

,id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."


In [105]:
# convert the tags data as there are all in list i want to make them as a string...
new_df['tags'] = new_df['tags'].apply(lambda x : " ".join(x))

In [106]:
# Sample Program explanation of this PorterStemmer

# from nltk.stem import PorterStemmer
# ps = PorterStemmer()
# words = ["love", "loving", "lovable", "loved", "happily", "running"]

# for w in words:
#     print(w , "→" , ps.stem(w))

# ==============OP================== 
# love → love
# loving → love
# lovable → lovabl
# loved → love
# happily → happili
# running → run

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()



In [107]:
def apply_stem(text):
    y = [] 
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

# We can also use Lemmatization

# apply_stem('In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron')
new_df['tags'] = new_df['tags'].apply(apply_stem)

In [108]:
new_df['tags'][0]

'in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization. action adventur fantasi sciencefict cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d samworthington zoesaldana sigourneyweav jamescameron'

In [109]:
# Converting into lower case
new_df['tags'] = new_df['tags'].apply(lambda x : x.lower())

**TEXT VECTORIZATION**

In [110]:
# We will not consider the stop words and then after removing that words we will select top 5000 words which is coming frequently..and then we will make a table of that 5000 words and then check movie by movie ..that how much time this word come in that so in that way we can understand which word come more frequent and we will get a table for each movie...

# text = ["I love movies", "I love programming"]
# x = cv.fit_transform(text)
# print(cv.get_feature_names_out())
# print(x.toarray())


cv = CountVectorizer(max_features=5000,stop_words='english')
vectors = cv.fit_transform(new_df['tags']).toarray()

In [111]:
vectors[0] # This is the vector for the first movie...
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      shape=(5000,), dtype=object)

In [112]:
# So here insted of Distance we are thinking to take the angle of the vector to find the similarity so that way it will be little helpfull..
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)

In [113]:
similarity[0] # So this is the similarity of first movie to all 4806 movies....so this is easy way to find the similarity..

array([1.        , 0.08346223, 0.0860309 , ..., 0.04499213, 0.        ,
       0.        ], shape=(4806,))

In [114]:
# Now Recomendation Function
def recommending_movie_list(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)) , reverse=True , key=lambda x : x[1])[1:6]
    
    for i in movies_list:
        print(new_df.iloc[i[0]].title)
    return 

recommending_movie_list('Batman Begins')

The Dark Knight
Batman
Batman
The Dark Knight Rises
10th & Wolf


In [115]:
import pickle
pickle.dump(new_df,open('movies.pkl' , 'wb'))

In [117]:
pickle.dump(similarity , open('similarity.pkl' , 'wb'))

In [116]:
pickle.dump(vectors , open("vector_metrics.pkl" , "wb"))